In [1]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'mimicivliza'  # Replace with your bucket name
mimic_iv_bhc = f's3://{bucket_name}/sample_data_100.csv'

# Load the files
mimic_iv_bhc_100 = pd.read_csv(mimic_iv_bhc)

# Display the data
mimic_iv_bhc_100.head(20)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,note_id,input,target,input_tokens,target_tokens
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...,Mr. ___ is a ___ yo man with CAD with prior MI...,3496,1143
2,12435705-DS-14,<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprof...,Mr. ___ is a ___ w/ Ph+ve ALL on dasatanib and...,5591,1098
3,12413577-DS-4,<SEX> F <SERVICE> OBSTETRICS/GYNECOLOGY <ALLER...,"On ___, Ms. ___ was admitted to the gynecology...",1119,221
4,17967161-DS-29,<SEX> M <SERVICE> SURGERY <ALLERGIES> lisinopr...,Mr. ___ underwent an angiogram on ___ which sh...,3307,439
5,16956007-DS-20,<SEX> M <SERVICE> SURGERY <ALLERGIES> Codeine ...,Mr. ___ is a ___ who underwent an exploratory ...,4168,1209
6,16919911-DS-15,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Penicil...,This is a ___ year old female with a recent di...,2059,208
7,15682570-DS-25,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ w/ h/o CAD ___ CABG LIMA to LAD, SVG to D1...",2215,451
8,12135369-DS-24,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Compazi...,Ms ___ is a ___ year old woman with a history ...,2132,416
9,11906321-DS-20,<SEX> M <SERVICE> NEUROSURGERY <ALLERGIES> Pat...,The patient was admitted to the neurosurgery s...,2347,316


In [2]:
# Install Hugging Face Transformers
!pip install transformers
!pip install sacremoses
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 157.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 161.2 MB/s eta 0:00:0000:0100:01


In [3]:
from huggingface_hub import login
import os

# Use token from environment variable (safer)
login(os.getenv("HF_TOKEN"))


In [4]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load configuration for 8-bit quantized inference
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True  # ✅ correctly placed inside config
)

# Define model ID for Mistral
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
).eval()


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
import torch
from tqdm import tqdm

# Few-shot examples for prompt construction (limit to 2 for better performance)
few_shot_examples = [
    {
        "input": "<SEX> F <SERVICE> ONCOLOGY <CHIEF COMPLAINT> worsening back pain <HISTORY OF PRESENT ILLNESS> The patient is a 45-year-old female with a history of metastatic breast cancer presenting with worsening back pain over the last two weeks...",
        "Summary": "A 45-year-old female with metastatic breast cancer presented with worsening back pain. Imaging showed thoracic spine fractures, and she received palliative radiation therapy."
    },
    {
        "input": "<SEX> M <SERVICE> CARDIOLOGY <CHIEF COMPLAINT> chest pain <HISTORY OF PRESENT ILLNESS> A 55-year-old male presented with chest pain radiating to his left arm and jaw...",
        "Summary": "A 55-year-old male with a myocardial infarction underwent PCI with stenting of the right coronary artery. He was monitored and started on dual antiplatelet therapy."
    }
]

# Function to construct the prompt for Mistral-7B with clearer separation
def construct_few_shot_prompt(input_text):
    prompt = (
    "You are a clinical summarization expert.\n"
    "Given a patient's structured case, summarize it in 2-3 sentences, including age, gender, primary complaint, diagnosis, and treatment.\n"
    "Your response must ONLY contain the summary in natural language, not tags or labels.\n\n")
    prompt += "DO NOT include any prompt text in your response, only provide the summary.\n\n"
    for example in few_shot_examples:
        prompt += f"### CASE:\n{example['input']}\n\n### SUMMARY:\n{example['Summary']}\n\n"
    prompt += f"### CASE:\n{input_text}\n\n### SUMMARY:"
    return f"<s>[INST] {prompt} [/INST]"

# Optional: process fewer rows to avoid memory crash
subset_df = mimic_iv_bhc_100.iloc[:101]  # ← You can change/remove this limit
generated_summaries = []

for input_text in tqdm(subset_df['input'], desc="Generating Summaries with Mistral-7B"):
    prompt = construct_few_shot_prompt(input_text)
    
    # Tokenize with truncation
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=4096, 
        truncation=True
    )
    
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    generation_params = {
        "do_sample": True,
        "top_p": 0.9, 
        "temperature": 0.7,
        "top_k": 50,
        "max_new_tokens": 250,
        "repetition_penalty": 1,
        "eos_token_id": tokenizer.eos_token_id
    }
    
    # Get the length of input tokens
    prompt_length = inputs["input_ids"].shape[1]
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            return_dict_in_generate=True,
            output_scores=False,
            **generation_params
        )
    
    # Extract only the new tokens
    new_tokens = outputs.sequences[0, prompt_length:]
    generated_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    
    # Comprehensive cleaning of any prompt formatting that might have been generated
    # Remove any prompt fragments
    for prefix in ["Input:", "Output:", "### CASE:", "### SUMMARY:", "CASE:", "SUMMARY:", "Case:", "Summary:"]:
        if prefix in generated_text:
            parts = generated_text.split(prefix)
            if prefix in ["### SUMMARY:", "SUMMARY:", "Summary:", "Output:"]:
                # Keep the part after the last occurrence
                generated_text = parts[-1].strip()
            else:
                # For other prefixes, keep only the part before
                generated_text = parts[0].strip()
    
    # Remove any examples text that might have been copied
    for example in few_shot_examples:
        if example['Summary'] in generated_text:
            generated_text = generated_text.replace(example['target'], "").strip()
    
    # Remove any instruction tags
    generated_text = generated_text.replace("[INST]", "").replace("[/INST]", "").strip()
    
    # Final check: If it still has Input/Case in it, take only the last paragraph
    if any(keyword in generated_text for keyword in ["Input:", "Case:", "Output:", "Summary:"]):
        paragraphs = generated_text.split('\n\n')
        generated_text = paragraphs[-1].strip()
    
    generated_summaries.append(generated_text)
    torch.cuda.empty_cache()  # clear memory after each example

# Save results
subset_df['generated_summary_mistral_7b'] = generated_summaries
subset_df.to_csv("mistral_7b_generated_summaries.csv", index=False)
print("✅ Summaries saved to 'mistral_7b_generated_summaries.csv'")

Generating Summaries with Mistral-7B: 100%|██████████| 100/100 [44:45<00:00, 26.86s/it]

✅ Summaries saved to 'mistral_7b_generated_summaries.csv'



/tmp/ipykernel_12008/3912003086.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['generated_summary_mistral_7b'] = generated_summaries


In [6]:
subset_df.head(4)

,note_id,input,target,input_tokens,target_tokens,generated_summary_mistral_7b
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75,A morbidly obese 45-year-old female with a his...
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...,Mr. ___ is a ___ yo man with CAD with prior MI...,3496,1143,Succinate XL 100 mg PO DAILY 20. HELD- meloxic...
2,12435705-DS-14,<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprof...,Mr. ___ is a ___ w/ Ph+ve ALL on dasatanib and...,5591,1098,IA ISOLATED. MICROBIOLOGY REPORT (Final ___: N...
3,12413577-DS-4,<SEX> F <SERVICE> OBSTETRICS/GYNECOLOGY <ALLER...,"On ___, Ms. ___ was admitted to the gynecology...",1119,221,A 45-year-old female with a history of ulcerat...


In [7]:
print(subset_df['input'].iloc[2])

<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprofen <ATTENDING> ___ <CHIEF COMPLAINT> Fever, altered mental status <MAJOR SURGICAL OR INVASIVE PROCEDURE> Parietal bone biopsy ___ <HISTORY OF PRESENT ILLNESS> ___ year old man with history of ruptured AVM on ___, s/p craniectomy/ embolization/ cranioplasty) who presented with abnormal CBC and was diagnosed with PH+ ALL and found to have a MRSA intracranial abscess/osteomyelitis at the site of his previous surgery in ___ of this year now with fever and altered mental status. The patient's wife told me that he had a fall on ___ when he got up to urinate but used his weak left hand to support himself. Has been altered since. Went to ___ but they dc-ed after negative NCHCT. However, presented again to ___ on ___ this time also had fevers. They placed foley (he uses diapers at baseline) and transferred him to ___. Received vanco/cefepime PTA. Hx and ROS is per wife. Pt is unable to contribute much. - In the ED, initial VS were 0 98.7 108 115/78 

In [8]:
print(subset_df['generated_summary_mistral_7b'].iloc[2])

IA ISOLATED. MICROBIOLOGY REPORT (Final ___: No growth in any of the cultures. No microorganisms isolated from CSF, parietal bone tissue, or anaerobic culture. MICROBIOLOGY REPORT (Final ___: No growth in any of the cultures. No microorganisms isolated from CSF, parietal bone tissue, or anaerobic culture. MICROBIOLOGY REPORT (Final ___: No growth in any of the cultures. No microorganisms isolated from CSF, parietal bone tissue, or anaerobic culture. MICROBIOLOGY REPORT (Final ___: No growth in any of the cultures. No microorganisms isolated from CSF, parietal bone tissue, or anaerobic culture. MICROBIOLOGY REPORT (Final ___: No growth in any of the cultures. No microorganisms isolated from CSF, parietal bone tissue, or anaerobic culture. MICROBIOLOGY REPORT (Final ___: No growth in any of the cultures. No microorganisms
